In [1]:
import numpy
import urllib
import scipy.optimize
import random
from collections import defaultdict
import nltk
import string
from nltk.stem.porter import *
from sklearn import linear_model
def parseData(fname):
    for l in urllib.request.urlopen(fname):
        yield eval(l)

### Just the first 5000 reviews

data = list(parseData("http://jmcauley.ucsd.edu/cse158/data/beer/beer_50000.json"))[:5000]

In [2]:
punctuation = set(string.punctuation)


In [3]:
old_w = ''
cob=''
cobCount = defaultdict(int)
for l in data:
    r = ''.join([c for c in l['review/text'].lower() if not c in punctuation])
    for w in r.split():
        if old_w != '':
            cob =old_w + ' '+ w
            judgement = True
        else:
            cob =''
            judgement = False
        if judgement == True:
            cobCount[cob] += 1    
            old_w = w
        else:
            old_w = w

In [4]:
counts = [(cobCount[w], w) for w in cobCount]
counts.sort()
counts.reverse()
counts[0:5]
#Question1  top 5 bigrams

[(4587, 'with a'),
 (2595, 'in the'),
 (2245, 'of the'),
 (2058, 'is a'),
 (2033, 'on the')]

In [5]:
#Qusetion2 

pop_bigrams = [x[1] for x in counts[:1000]]
bigramsId = dict(zip(pop_bigrams, range(len(pop_bigrams))))
bigramsSet = set(pop_bigrams)
def feature(datum):
    old_w = ''
    cob=''
    feat = [0]*len(pop_bigrams)
    r = ''.join([c for c in datum['review/text'].lower() if not c in punctuation])
    for w in r.split():
        if old_w != '':
            cob =old_w + ' '+ w
            judgement = True
        else:
            cob =''
            judgement = False
        if judgement == True:
            cobCount[cob] += 1    
            old_w = w
        else:
            old_w = w
        if cob in pop_bigrams:
            feat[bigramsId[cob]] += 1
    feat.append(1) #offset
    return feat

X = [feature(d) for d in data]
y = [d['review/overall'] for d in data]


In [6]:
clf = linear_model.Ridge(1.0, fit_intercept=False)
clf.fit(X, y)
theta = clf.coef_
predictions = clf.predict(X)

In [7]:
s= 0
for i in range(len(y)):
    s +=(y[i]-predictions[i])**2
MSE = s/len(y)
MSE

0.3430153408719768

In [8]:
#Question 3
review_list = []
for datum in data:
    r = ''.join([c for c in datum['review/text'].lower() if not c in punctuation])
    temporary=''
    for w in r.split():
        temporary += w +' '
    review_list.append(temporary)

In [9]:
n = len(review_list)
n

5000

In [10]:
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in data:
  r = ''.join([c for c in d['review/text'].lower() if not c in punctuation])
  for w in r.split():
    wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

words = [x[1] for x in counts[:1000]]

In [11]:
import math
count_foam= 0
for i in range(len(review_list)):
    if 'foam' in review_list[i].split():
        count_foam+= 1
count_smell= 0
for i in range(len(review_list)):
    if 'smell' in review_list[i].split():
        count_smell+= 1
count_banana= 0
for i in range(len(review_list)):
    if 'banana' in review_list[i].split():
        count_banana+= 1
count_lactic= 0
for i in range(len(review_list)):
    if 'lactic' in review_list[i].split():
        count_lactic+= 1
count_tart= 0
for i in range(len(review_list)):
    if 'tart' in review_list[i].split():
        count_tart+= 1
IDF_foam  = math.log10(n/(count_foam))
IDF_smell  = math.log10(n/(count_smell))
IDF_banana  = math.log10(n/(count_banana))
IDF_lactic  = math.log10(n/(count_lactic))
IDF_tart  = math.log10(n/(count_tart))
print('foam',IDF_foam)
print('smell',IDF_smell)
print('banana',IDF_banana)
print('lactic',IDF_lactic)
print('tart',IDF_tart)

foam 1.1378686206869628
smell 0.5379016188648442
banana 1.6777807052660807
lactic 2.9208187539523753
tart 1.8068754016455384


In [12]:
TF_foam = 0
TF_smell = 0
TF_banana = 0
TF_lactic = 0
TF_tart = 0
datum = data[0]
r = ''.join([c for c in datum['review/text'].lower() if not c in punctuation])
for w in r.split():
    if w =='foam':
        TF_foam += 1
    if w =='smell':
        TF_smell+=1
    if w =='banana':
        TF_banana+=1
    if w =='lactic':
        TF_lactic+=1
    if w =='tart':
        TF_tart+=1


In [13]:
tf_idf_foam = TF_foam *IDF_foam
tf_idf_smell = TF_smell *IDF_smell
tf_idf_banana = TF_banana *IDF_banana
tf_idf_lactic = TF_lactic *IDF_lactic
tf_idf_tart = TF_tart *IDF_tart
print('foam',tf_idf_foam)
print('smell',tf_idf_smell)
print('banana',tf_idf_banana)
print('lactic',tf_idf_lactic)
print('tart',tf_idf_tart)

foam 2.2757372413739256
smell 0.5379016188648442
banana 3.3555614105321614
lactic 5.841637507904751
tart 1.8068754016455384


In [15]:
#question4 
idf_set =defaultdict(int)
count = 0
indicator = 0
for i in words:
    for j in range(len(review_list)):
        if i in set(review_list[j].split()):
            count+= 1
    idf_set[i] = count
    indicator+=1
    count=0

In [16]:
for i in idf_set:
    idf_set[i]=math.log10(n/idf_set[i])

In [17]:
tf_set_r1 = defaultdict(int)
tf_set_r2 = defaultdict(int)
for i in idf_set:
    tf_set_r1[i]=0
for i in idf_set:
    tf_set_r2[i]=0
datum1 = data[0]
datum2 = data[1]
r1 = ''.join([c for c in datum1['review/text'].lower() if not c in punctuation])
r2 = ''.join([c for c in datum2['review/text'].lower() if not c in punctuation])
for w in r1.split():
    for i in idf_set:
        if i == w:
            tf_set_r1[i]+=1
for w in r2.split():
    for i in idf_set:
        if i == w:
            tf_set_r2[i]+=1

In [18]:
tf_idf_r1= {}
tf_idf_r2= {}

for i in idf_set:
    tf_idf_r1[i] = tf_set_r1[i]*idf_set[i]
for i in idf_set:
    tf_idf_r2[i] = tf_set_r2[i]*idf_set[i]

In [19]:
#cosine similarity
p= 0
k= 0
d= 0
for i in tf_idf_r1:
    p +=tf_idf_r1[i] *tf_idf_r2[i]
    k +=tf_idf_r1[i]**2 
    d +=tf_idf_r2[i]**2 
cos= p/((k**0.5)*(d**0.5))
cos

0.10613024167865802

In [20]:
import numpy as np
def cosine_similarity(vector1, vector2):
    try:
        m = np.array(vector1).T
        n = np.array(vector2).T
        p= sum(m*n)
        k = (sum(m*m)**0.5)*(sum(n*n)**0.5)
        return p/k
    except:
        return 0

In [21]:
x=[]
for i in range(0,11):
    x.append(i*500)


In [22]:
#question 5
tf_set_r = defaultdict(int)
for i in range(0,5000):
    tf_set_r[i]=defaultdict(int)
    for k in idf_set:
        tf_set_r[i][k]=0
    if i in x:
        print(i)

0
500
1000
1500
2000
2500
3000
3500
4000
4500


In [23]:
for i in range(len(data)):
    r = ''.join([c for c in data[i]['review/text'].lower() if not c in punctuation])
    for w in r.split():
        for j in idf_set:
            if j == w:
                tf_set_r[i][j]+=1
    if i in x:
        print(i)

0
500
1000
1500
2000
2500
3000
3500
4000
4500


In [24]:
tf_idf_r = defaultdict(int)
for i in range(0,5000):
    tf_idf_r[i] =[]
for k in range(len(tf_set_r)):
    for i in idf_set:
        tf_idf_r[k].append(tf_set_r[k][i]*idf_set[i])

In [25]:
cos_dict={}
for i in range(len(tf_idf_r)):
    cos_dict[i] = cosine_similarity(tf_idf_r[0],tf_idf_r[i])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  


In [26]:
cos_dict[0]=0
highest_pair = max(cos_dict,key=cos_dict.get)
max_data = data[highest_pair]
max_text = ''.join([c for c in max_data['review/text'].lower() if not c in punctuation])
max_beerID= max_data[ 'beer/beerId']
print(max_beerID)
print(max_text)


52211
poured from a 22oz bottle to a dogfish head snifter		color slight hazy orange with an off white head		smell cinnamon banana pumpkin and nutmeg		taste alcohol pumpkin nutmeg allspice and a hint of banana		mouthfeel medium carbonation smooth medium dryness on the palate		overall the smell is great the banana was a huge surprise for me the taste had too much alcohol presence seemed to overpower the other flavors cheers


In [27]:
temp_list=[]
new_list = []
for i in range(len(tf_idf_r)):
    temp_list.append(1)
    for j in range(len(tf_idf_r[i])):
        temp_list.append(tf_idf_r[i][j])
    new_list.append(temp_list)
    temp_list=[]


In [28]:
#question 6

X = new_list
y = [d['review/overall'] for d in data]
clf = linear_model.Ridge(1.0, fit_intercept=False)
clf.fit(X, y)
theta = clf.coef_
predictions = clf.predict(X)


In [29]:
s= 0
for i in range(len(y)):
    s +=(y[i]-predictions[i])**2
MSE = s/len(y)
MSE

0.27875956007772235

In [3]:
#question7
import random
data_final = list(parseData("http://jmcauley.ucsd.edu/cse158/data/beer/beer_50000.json"))[:15000]
random.shuffle(data_final)
data_train = data_final[0:5000]
data_validation = data_final[5000:10000]
data_test = data_final[10000:15000]

In [196]:
#bigrams-removing-counts situation 1
punctuation = set(string.punctuation)
old_w = ''
cob=''
cobCount = defaultdict(int)
for l in data_train:
    r = ''.join([c for c in l['review/text'].lower() if not c in punctuation])
    for w in r.split():
        if old_w != '':
            cob =old_w + ' '+ w
            judgement = True
        else:
            cob =''
            judgement = False
        if judgement == True:
            cobCount[cob] += 1    
            old_w = w
        else:
            old_w = w
counts1 = [(cobCount[w], w) for w in cobCount]
counts1.sort()
counts1.reverse()

In [197]:
#validation to get the regularization
regularization =[0.01,0.1,1,10,100]
pop_bigrams = [x[1] for x in counts1[:1000]]
bigramsId = dict(zip(pop_bigrams, range(len(pop_bigrams))))
bigramsSet = set(pop_bigrams)
def feature(datum):
    old_w = ''
    cob=''
    feat = [0]*len(pop_bigrams)
    r = ''.join([c for c in datum['review/text'].lower() if not c in punctuation])
    for w in r.split():
        if old_w != '':
            cob =old_w + ' '+ w
            judgement = True
        else:
            cob =''
            judgement = False
        if judgement == True:
            cobCount[cob] += 1    
            old_w = w
        else:
            old_w = w
        if cob in pop_bigrams:
            feat[bigramsId[cob]] += 1
    feat.append(1) #offset
    return feat

X = [feature(d) for d in data_train]
y = [d['review/overall'] for d in data_train]
X_validation =[feature(d) for d in data_validation]
y_validation = [d['review/overall'] for d in data_validation]
for k in regularization : 
    clf = linear_model.Ridge(k, fit_intercept=False)
    clf.fit(X, y)
    theta = clf.coef_
    predictions = clf.predict(X_validation)
    s= 0
    for i in range(len(y_validation)):
        s +=(y_validation[i]-predictions[i])**2
    MSE = s/len(y)
    print(k,MSE)


0.01 0.518359500390911
0.1 0.5180331915851174
1 0.5149059632746981
10 0.493387226900679
100 0.48356826924084745


In [202]:
    #test,using 100
    X = [feature(d) for d in data_train]
    y = [d['review/overall'] for d in data_train]
    X_test = [feature(d) for d in data_test]
    y_test = [d['review/overall'] for d in data_test]

    clf = linear_model.Ridge(10, fit_intercept=False)
    clf.fit(X, y)
    theta = clf.coef_
    predictions = clf.predict(X_test)
    s= 0
    for i in range(len(y_test)):
        s +=(y_test[i]-predictions[i])**2
    MSE = s/len(y)
    print('100',MSE)

100 0.493292372538536


In [203]:
#unigrams-removing-counts situation 2
punctuation = set(string.punctuation)
old_w = ''
cob=''
cobCount = defaultdict(int)
for l in data_train:
    r = ''.join([c for c in l['review/text'].lower() if not c in punctuation])
    for w in r.split():
        wordCount[w] += 1
counts2 = [(wordCount[w], w) for w in wordCount]
counts2.sort()
counts2.reverse()

In [204]:
#validation to get the regularization
regularization =[0.01,0.1,1,10,100]
pop_unigrams = [x[1] for x in counts2[:1000]]

wordId = dict(zip(pop_unigrams, range(len(pop_unigrams))))
wordSet = set(pop_unigrams)

def feature(datum):
  feat = [0]*len(pop_unigrams)
  r = ''.join([c for c in datum['review/text'].lower() if not c in punctuation])
  for w in r.split():
    if w in pop_unigrams:
      feat[wordId[w]] += 1
  feat.append(1) #offset
  return feat

X = [feature(d) for d in data_train]
y = [d['review/overall'] for d in data_train]
X_validation =[feature(d) for d in data_validation]
y_validation = [d['review/overall'] for d in data_validation]
for k in regularization : 
    clf = linear_model.Ridge(k, fit_intercept=False)
    clf.fit(X, y)
    theta = clf.coef_
    predictions = clf.predict(X_validation)
    s= 0
    for i in range(len(y_validation)):
        s +=(y_validation[i]-predictions[i])**2
    MSE = s/len(y)
    print(k,MSE)


0.01 0.4529409338050712
0.1 0.452668634353386
1 0.4500457654993608
10 0.4311986124477603
100 0.4289422861179684


In [205]:
    #test  k =100
    X = [feature(d) for d in data_train]
    y = [d['review/overall'] for d in data_train]
    X_test = [feature(d) for d in data_test]
    y_test = [d['review/overall'] for d in data_test]

    clf = linear_model.Ridge(10, fit_intercept=False)
    clf.fit(X, y)
    theta = clf.coef_
    predictions = clf.predict(X_test)
    s= 0
    for i in range(len(y_test)):
        s +=(y_test[i]-predictions[i])**2
    MSE = s/len(y)
    print('100',MSE)

100 0.42691586891007255


In [63]:
#unigrams-removing-tfidf situation 3
review_list = []
data = data_train
for datum in data:
    r = ''.join([c for c in datum['review/text'].lower() if not c in punctuation])
    temporary=''
    for w in r.split():
        temporary += w +' '
    review_list.append(temporary)
    
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in data:
  r = ''.join([c for c in d['review/text'].lower() if not c in punctuation])
  for w in r.split():
    wordCount[w] += 1

counts3 = [(wordCount[w], w) for w in wordCount]
counts3.sort()
counts3.reverse()

words = [x[1] for x in counts3[:1000]]

x=[100,200,300,400,500,600,700,800,900]


idf_set =defaultdict(int)
count = 0
indicator = 0
for i in words:
    for j in range(len(review_list)):
        if i in set(review_list[j].split()):
            count+= 1
    idf_set[i] = count
    indicator+=1
    count=0
    if indicator in x:
        print(indicator)

100
200
300
400
500
600
700
800
900


In [9]:
import math
n=5000
for i in idf_set:
    idf_set[i]=math.log10(n/idf_set[i])
tf_set_r_train = defaultdict(int)
for i in range(0,5000):
    tf_set_r_train[i]=defaultdict(int)
    for k in idf_set:
        tf_set_r_train[i][k]=0
for i in range(len(data)):
    r = ''.join([c for c in data_train[i]['review/text'].lower() if not c in punctuation])
    for w in r.split():
        for j in idf_set:
            if j == w:
                tf_set_r_train[i][j]+=1
tf_idf_r_train = defaultdict(int)
for i in range(0,5000):
    tf_idf_r_train[i] =[]
for k in range(len(tf_set_r_train)):
    for i in idf_set:
        tf_idf_r_train[k].append(tf_set_r_train[k][i]*idf_set[i])
tf_set_r_validation = defaultdict(int)
for i in range(0,5000):
    tf_set_r_validation[i]=defaultdict(int)
    for k in idf_set:
        tf_set_r_validation[i][k]=0
for i in range(len(data)):
    r = ''.join([c for c in data_validation[i]['review/text'].lower() if not c in punctuation])
    for w in r.split():
        for j in idf_set:
            if j == w:
                tf_set_r_validation[i][j]+=1
tf_idf_r_validation = defaultdict(int)
for i in range(0,5000):
    tf_idf_r_validation[i] =[]
for k in range(len(tf_set_r_validation)):
    for i in idf_set:
        tf_idf_r_validation[k].append(tf_set_r_validation[k][i]*idf_set[i])
tf_set_r_test = defaultdict(int)
for i in range(0,5000):
    tf_set_r_test[i]=defaultdict(int)
    for k in idf_set:
        tf_set_r_test[i][k]=0
for i in range(len(data)):
    r = ''.join([c for c in data_test[i]['review/text'].lower() if not c in punctuation])
    for w in r.split():
        for j in idf_set:
            if j == w:
                tf_set_r_test[i][j]+=1
tf_idf_r_test = defaultdict(int)
for i in range(0,5000):
    tf_idf_r_test[i] =[]
for k in range(len(tf_set_r_test)):
    for i in idf_set:
        tf_idf_r_test[k].append(tf_set_r_test[k][i]*idf_set[i])

In [16]:
temp_list_train=[]
new_list_train = []
for i in range(len(tf_idf_r_train)):
    temp_list_train.append(1)
    for j in range(len(tf_idf_r_train[i])):
        temp_list_train.append(tf_idf_r_train[i][j])
    new_list_train.append(temp_list_train)
    temp_list_train=[]
X_train = new_list_train
y_train = [d['review/overall'] for d in data_train]

temp_list_validation=[]
new_list_validation = []
for i in range(len(tf_idf_r_validation)):
    temp_list_validation.append(1)
    for j in range(len(tf_idf_r_validation[i])):
        temp_list_validation.append(tf_idf_r_validation[i][j])
    new_list_validation.append(temp_list_validation)
    temp_list_validation=[]
X_validation = new_list_validation
y_validation = [d['review/overall'] for d in data_validation]

temp_list_test=[]
new_list_test = []
for i in range(len(tf_idf_r_test)):
    temp_list_test.append(1)
    for j in range(len(tf_idf_r_test[i])):
        temp_list_test.append(tf_idf_r_test[i][j])
    new_list_test.append(temp_list_test)
    temp_list_test=[]
X_test = new_list_test
y_test = [d['review/overall'] for d in data_test]


In [19]:
regularization =[0.01,0.1,1,10,100]
for k in regularization : 
    clf = linear_model.Ridge(k, fit_intercept=False)
    clf.fit(X_train, y_train)
    theta = clf.coef_
    predictions = clf.predict(X_validation)
    s= 0
    for i in range(len(y_validation)):
        s +=(y_validation[i]-predictions[i])**2
    MSE = s/len(y_validation)
    print(k,MSE)

0.01 0.42938963546198694
0.1 0.4293010105047813
1 0.42849054310134843
10 0.42249042777884943
100 0.4336330459164157


In [20]:
    #best k is 10
    clf = linear_model.Ridge(10, fit_intercept=False)
    clf.fit(X_train, y_train)
    theta = clf.coef_
    predictions = clf.predict(X_test)
    s= 0
    for i in range(len(y_test)):
        s +=(y_test[i]-predictions[i])**2
    MSE = s/len(y_test)
    print(10,MSE)

10 0.4461932505297838


In [43]:
#biograms-removing-tfidf situation 4

old_w = ''
cob=''
cobCount = defaultdict(int)
for d in data:
    r = ''.join([c for c in d['review/text'].lower() if not c in punctuation])
    for w in r.split():
        if old_w != '':
            cob =old_w + ' '+ w
            judgement = True
        else:
            cob =''
            judgement = False
        if judgement == True:
            cobCount[cob] += 1    
            old_w = w
        else:
            old_w = w

counts4 = [(cobCount[w], w) for w in cobCount]
counts4.sort()
counts4.reverse()

words = [x[1] for x in counts4[:1000]]





In [48]:
idf_set =defaultdict(int)
count = 0
indicator = 0
x =[10,20,50,100,200,300,400,500,600,700,800,900]
for i in words:
    for j in range(len(review_list)):
        old_w = ''
        cob=''
        cobCount = []
        for w in review_list[j].split():
            if old_w != '':
                cob =old_w + ' '+ w
                judgement = True
            else :
                cob =''
                judgement = False
            if judgement == True:
                cobCount.append(cob)    
                old_w = w
            else:
                old_w = w
        for k in cobCount:
            if k == i:
                idf_set[i]+= 1
for i in idf_set:
idf_set[i]=math.log10(n/(idf_set[i]+1))

In [54]:

tf_set_r_train = defaultdict(int)
for i in range(0,5000):
    tf_set_r_train[i]=defaultdict(int)
    for k in idf_set:
        tf_set_r_train[i][k]=0
old_w = ''
cob=''
cobCount = defaultdict(int)
for i in range(len(data)):
    r = ''.join([c for c in data_train[i]['review/text'].lower() if not c in punctuation])
    for w in r.split():
        if old_w != '':
            cob =old_w + ' '+ w
            judgement = True
            old_w = w
        else:
            cob =''  
            old_w = w
        for j in idf_set:
            if j == cob:
                tf_set_r_train[i][j]+=1
tf_idf_r_train = defaultdict(int)
for i in range(0,5000):
    tf_idf_r_train[i] =[]
for k in range(len(tf_set_r_train)):
    for i in idf_set:
        tf_idf_r_train[k].append(tf_set_r_train[k][i]*idf_set[i])


In [57]:
tf_set_r_validation = defaultdict(int)
for i in range(0,5000):
    tf_set_r_validation[i]=defaultdict(int)
    for k in idf_set:
        tf_set_r_validation[i][k]=0
old_w = ''
cob=''
cobCount = defaultdict(int)
for i in range(len(data)):
    r = ''.join([c for c in data_validation[i]['review/text'].lower() if not c in punctuation])
    for w in r.split():
        if old_w != '':
            cob =old_w + ' '+ w
            judgement = True
            old_w = w
        else:
            cob =''  
            old_w = w
        for j in idf_set:
            if j == cob:
                tf_set_r_validation[i][j]+=1
tf_idf_r_validation = defaultdict(int)
for i in range(0,5000):
    tf_idf_r_validation[i] =[]
for k in range(len(tf_set_r_validation)):
    for i in idf_set:
        tf_idf_r_validation[k].append(tf_set_r_validation[k][i]*idf_set[i])
old_w = ''
cob=''
cobCount = defaultdict(int)
tf_set_r_test = defaultdict(int)
for i in range(0,5000):
    tf_set_r_test[i]=defaultdict(int)
    for k in idf_set:
        tf_set_r_test[i][k]=0
for i in range(len(data)):
    r = ''.join([c for c in data_test[i]['review/text'].lower() if not c in punctuation])
    for w in r.split():
        if old_w != '':
            cob =old_w + ' '+ w
            judgement = True
            old_w = w
        else:
            cob =''  
            old_w = w
        for j in idf_set:
            if j == cob:
                tf_set_r_test[i][j]+=1
tf_idf_r_test = defaultdict(int)
for i in range(0,5000):
    tf_idf_r_test[i] =[]
for k in range(len(tf_set_r_test)):
    for i in idf_set:
        tf_idf_r_test[k].append(tf_set_r_test[k][i]*idf_set[i])

In [58]:
temp_list_train=[]
new_list_train = []
for i in range(len(tf_idf_r_train)):
    temp_list_train.append(1)
    for j in range(len(tf_idf_r_train[i])):
        temp_list_train.append(tf_idf_r_train[i][j])
    new_list_train.append(temp_list_train)
    temp_list_train=[]
X_train = new_list_train
y_train = [d['review/overall'] for d in data_train]

temp_list_validation=[]
new_list_validation = []
for i in range(len(tf_idf_r_validation)):
    temp_list_validation.append(1)
    for j in range(len(tf_idf_r_validation[i])):
        temp_list_validation.append(tf_idf_r_validation[i][j])
    new_list_validation.append(temp_list_validation)
    temp_list_validation=[]
X_validation = new_list_validation
y_validation = [d['review/overall'] for d in data_validation]

temp_list_test=[]
new_list_test = []
for i in range(len(tf_idf_r_test)):
    temp_list_test.append(1)
    for j in range(len(tf_idf_r_test[i])):
        temp_list_test.append(tf_idf_r_test[i][j])
    new_list_test.append(temp_list_test)
    temp_list_test=[]
X_test = new_list_test
y_test = [d['review/overall'] for d in data_test]

In [59]:
regularization =[0.01,0.1,1,10,100]
for k in regularization : 
    clf = linear_model.Ridge(k, fit_intercept=False)
    clf.fit(X_train, y_train)
    theta = clf.coef_
    predictions = clf.predict(X_validation)
    s= 0
    for i in range(len(y_validation)):
        s +=(y_validation[i]-predictions[i])**2
    MSE = s/len(y_validation)
    print(k,MSE)

0.01 0.5016421359342702
0.1 0.5015109865034666
1 0.5005300661904458
10 0.49339567675744767
100 0.49246433182787


In [60]:
    #best k is 100
    clf = linear_model.Ridge(100, fit_intercept=False)
    clf.fit(X_train, y_train)
    theta = clf.coef_
    predictions = clf.predict(X_test)
    s= 0
    for i in range(len(y_test)):
        s +=(y_test[i]-predictions[i])**2
    MSE = s/len(y_test)
    print(100,MSE)

100 0.5195463603282625


In [93]:
#biograms-not removing-tfidf situation 5
data = data_train
import nltk
old_w = ''
cob=''
cobCount = defaultdict(int)
for d in data:
    r = ''.join([c for c in d['review/text'].lower()]) 
    for w in nltk.word_tokenize(r):
        cob =old_w + ' '+ w
        judgement = True
        if judgement == True:
            cobCount[cob] += 1    
            old_w = w
        else:
            old_w = w

counts5 = [(cobCount[w], w) for w in cobCount]
counts5.sort()
counts5.reverse()

words = [x[1] for x in counts5[:1000]]


In [96]:
idf_set =defaultdict(int)
count = 0
indicator = 0
x =[10,20,50,100,200,300,400,500,600,700,800,900]
for i in words:
    for j in range(len(review_list)):
        old_w = ''
        cob=''
        cobCount = []
        for w in nltk.word_tokenize(review_list[j]):
            cob =old_w + ' '+ w
            judgement = True
            if judgement == True:
                cobCount.append(cob)    
                old_w = w
            else:
                old_w = w
        for k in cobCount:
            if k == i:
                idf_set[i]+= 1
for i in idf_set:
    idf_set[i]=math.log10(n/(idf_set[i]+1))

KeyboardInterrupt: 

In [73]:
tf_set_r_train = defaultdict(int)
for i in range(0,5000):
    tf_set_r_train[i]=defaultdict(int)
    for k in idf_set:
        tf_set_r_train[i][k]=0
old_w = ''
cob=''
cobCount = defaultdict(int)
for i in range(len(data)):
    r = ''.join([c for c in data_train[i]['review/text'].lower() ])
    for w in nltk.word_tokenize(r):
        cob =old_w + ' '+ w
        judgement = True
        old_w = w
        for j in idf_set:
            if j == cob:
                tf_set_r_train[i][j]+=1
tf_idf_r_train = defaultdict(int)
for i in range(0,5000):
    tf_idf_r_train[i] =[]
for k in range(len(tf_set_r_train)):
    for i in idf_set:
        tf_idf_r_train[k].append(tf_set_r_train[k][i]*idf_set[i])

In [77]:
tf_set_r_validation = defaultdict(int)
for i in range(0,5000):
    tf_set_r_validation[i]=defaultdict(int)
    for k in idf_set:
        tf_set_r_validation[i][k]=0
old_w = ''
cob=''
cobCount = defaultdict(int)
for i in range(len(data)):
    r = ''.join([c for c in data_validation[i]['review/text'].lower() ])
    for w in nltk.word_tokenize(r):
        cob =old_w + ' '+ w
        judgement = True
        old_w = w
        for j in idf_set:
            if j == cob:
                tf_set_r_validation[i][j]+=1
tf_idf_r_validation = defaultdict(int)
for i in range(0,5000):
    tf_idf_r_validation[i] =[]
for k in range(len(tf_set_r_validation)):
    for i in idf_set:
        tf_idf_r_validation[k].append(tf_set_r_validation[k][i]*idf_set[i])

In [80]:
tf_set_r_test = defaultdict(int)
for i in range(0,5000):
    tf_set_r_test[i]=defaultdict(int)
    for k in idf_set:
        tf_set_r_test[i][k]=0
old_w = ''
cob=''
cobCount = defaultdict(int)
for i in range(len(data)):
    r = ''.join([c for c in data_test[i]['review/text'].lower() ])
    for w in nltk.word_tokenize(r):
        cob =old_w + ' '+ w
        judgement = True
        old_w = w
        for j in idf_set:
            if j == cob:
                tf_set_r_test[i][j]+=1
tf_idf_r_test = defaultdict(int)
for i in range(0,5000):
    tf_idf_r_test[i] =[]
for k in range(len(tf_set_r_test)):
    for i in idf_set:
        tf_idf_r_test[k].append(tf_set_r_test[k][i]*idf_set[i])

In [81]:
temp_list_train=[]
new_list_train = []
for i in range(len(tf_idf_r_train)):
    temp_list_train.append(1)
    for j in range(len(tf_idf_r_train[i])):
        temp_list_train.append(tf_idf_r_train[i][j])
    new_list_train.append(temp_list_train)
    temp_list_train=[]
X_train = new_list_train
y_train = [d['review/overall'] for d in data_train]

temp_list_validation=[]
new_list_validation = []
for i in range(len(tf_idf_r_validation)):
    temp_list_validation.append(1)
    for j in range(len(tf_idf_r_validation[i])):
        temp_list_validation.append(tf_idf_r_validation[i][j])
    new_list_validation.append(temp_list_validation)
    temp_list_validation=[]
X_validation = new_list_validation
y_validation = [d['review/overall'] for d in data_validation]

temp_list_test=[]
new_list_test = []
for i in range(len(tf_idf_r_test)):
    temp_list_test.append(1)
    for j in range(len(tf_idf_r_test[i])):
        temp_list_test.append(tf_idf_r_test[i][j])
    new_list_test.append(temp_list_test)
    temp_list_test=[]
X_test = new_list_test
y_test = [d['review/overall'] for d in data_test]

In [82]:
regularization =[0.01,0.1,1,10,100]
for k in regularization : 
    clf = linear_model.Ridge(k, fit_intercept=False)
    clf.fit(X_train, y_train)
    theta = clf.coef_
    predictions = clf.predict(X_validation)
    s= 0
    for i in range(len(y_validation)):
        s +=(y_validation[i]-predictions[i])**2
    MSE = s/len(y_validation)
    print(k,MSE)

0.01 0.5029736634134457
0.1 0.5029452188735911
1 0.5026438855207455
10 0.49963811353783966
100 0.5088605762376635


In [83]:
    #best k is 10
    clf = linear_model.Ridge(10, fit_intercept=False)
    clf.fit(X_train, y_train)
    theta = clf.coef_
    predictions = clf.predict(X_test)
    s= 0
    for i in range(len(y_test)):
        s +=(y_test[i]-predictions[i])**2
    MSE = s/len(y_test)
    print(10,MSE)

10 0.49037692103212255


In [4]:
#uniograms-not removing-tfidf situation 6
review_list = []
data = data_train

for datum in data:
    r = ''.join([c for c in datum['review/text'].lower() ])
    temporary=''
    for w in nltk.word_tokenize(r):
        temporary += w +' '
    review_list.append(temporary)
    
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in data:
  r = ''.join([c for c in d['review/text'].lower() ])
  for w in nltk.word_tokenize(r):
    wordCount[w] += 1

counts6 = [(wordCount[w], w) for w in wordCount]
counts6.sort()
counts6.reverse()

words = [x[1] for x in counts6[:1000]]


In [5]:
#It is too slow to run oh my god
idf_set =defaultdict(int)
count = 0
indicator = 0
for i in words:
    for j in range(len(review_list)):
        old_w = ''
        cob=''
        cobCount = []
        for w in nltk.word_tokenize(review_list[j]):
            cob =old_w + ' '+ w
            judgement = True
            if judgement == True:
                cobCount.append(cob)    
                old_w = w
            else:
                old_w = w
        for k in cobCount:
            if k == i:
                idf_set[i]+= 1
for i in idf_set:
    idf_set[i]=math.log10(n/(idf_set[i]+1))

KeyboardInterrupt: 

In [ ]:
tf_set_r_train = defaultdict(int)
for i in range(0,5000):
    tf_set_r_train[i]=defaultdict(int)
    for k in idf_set:
        tf_set_r_train[i][k]=0
old_w = ''
cob=''
cobCount = defaultdict(int)
for i in range(len(data)):
    r = ''.join([c for c in data_train[i]['review/text'].lower() ])
    for w in nltk.word_tokenize(r):
        cob =old_w + ' '+ w
        judgement = True
        old_w = w
        for j in idf_set:
            if j == cob:
                tf_set_r_train[i][j]+=1
tf_idf_r_train = defaultdict(int)
for i in range(0,5000):
    tf_idf_r_train[i] =[]
for k in range(len(tf_set_r_train)):
    for i in idf_set:
        tf_idf_r_train[k].append(tf_set_r_train[k][i]*idf_set[i])

In [109]:
#biograms-not removing-counts situation 7
old_w = ''
cob=''
cobCount = defaultdict(int)
for l in data_train:
    r = ''.join([c for c in l['review/text'].lower()])
    for w in nltk.word_tokenize(r):
        if old_w != '':
            cob =old_w + ' '+ w
            judgement = True
        else:
            cob =''
            judgement = False
        if judgement == True:
            cobCount[cob] += 1    
            old_w = w
        else:
            old_w = w
counts7 = [(cobCount[w], w) for w in cobCount]
counts7.sort()
counts7.reverse()


In [110]:
regularization =[0.01,0.1,1,10,100]
pop_bigrams = [x[1] for x in counts7[:1000]]
bigramsId = dict(zip(pop_bigrams, range(len(pop_bigrams))))
bigramsSet = set(pop_bigrams)
def feature(datum):
    old_w = ''
    cob=''
    feat = [0]*len(pop_bigrams)
    r = ''.join([c for c in datum['review/text'].lower() ])
    for w in nltk.word_tokenize(r):
        if old_w != '':
            cob =old_w + ' '+ w
            judgement = True
        else:
            cob =''
            judgement = False
        if judgement == True:
            cobCount[cob] += 1    
            old_w = w
        else:
            old_w = w
        if cob in pop_bigrams:
            feat[bigramsId[cob]] += 1
    feat.append(1) #offset
    return feat

X = [feature(d) for d in data_train]
y = [d['review/overall'] for d in data_train]
X_validation =[feature(d) for d in data_validation]
y_validation = [d['review/overall'] for d in data_validation]
for k in regularization : 
    clf = linear_model.Ridge(k, fit_intercept=False)
    clf.fit(X, y)
    theta = clf.coef_
    predictions = clf.predict(X_validation)
    s= 0
    for i in range(len(y_validation)):
        s +=(y_validation[i]-predictions[i])**2
    MSE = s/len(y)
    print(k,MSE)

0.01 0.5222570758086934
0.1 0.5220068594032546
1 0.5196136444777321
10 0.5027507215657296
100 0.4978533182387553


In [111]:
    #test,using 100
    X = [feature(d) for d in data_train]
    y = [d['review/overall'] for d in data_train]
    X_test = [feature(d) for d in data_test]
    y_test = [d['review/overall'] for d in data_test]

    clf = linear_model.Ridge(100, fit_intercept=False)
    clf.fit(X, y)
    theta = clf.coef_
    predictions = clf.predict(X_test)
    s= 0
    for i in range(len(y_test)):
        s +=(y_test[i]-predictions[i])**2
    MSE = s/len(y)
    print('100',MSE)

100 0.48003136101779


In [107]:
#uiograms-not removing-counts situation 8
old_w = ''
cob=''
cobCount = defaultdict(int)
for l in data_train:
    r = ''.join([c for c in l['review/text'].lower() ])
    for w in nltk.word_tokenize(r):
        wordCount[w] += 1
counts8 = [(wordCount[w], w) for w in wordCount]
counts8.sort()
counts8.reverse()
regularization =[0.01,0.1,1,10,100]
pop_unigrams = [x[1] for x in counts8[:1000]]

wordId = dict(zip(pop_unigrams, range(len(pop_unigrams))))
wordSet = set(pop_unigrams)

def feature(datum):
  feat = [0]*len(pop_unigrams)
  r = ''.join([c for c in datum['review/text'].lower() ])
  for w in nltk.word_tokenize(r):
    if w in pop_unigrams:
      feat[wordId[w]] += 1
  feat.append(1) #offset
  return feat

X = [feature(d) for d in data_train]
y = [d['review/overall'] for d in data_train]
X_validation =[feature(d) for d in data_validation]
y_validation = [d['review/overall'] for d in data_validation]
for k in regularization : 
    clf = linear_model.Ridge(k, fit_intercept=False)
    clf.fit(X, y)
    theta = clf.coef_
    predictions = clf.predict(X_validation)
    s= 0
    for i in range(len(y_validation)):
        s +=(y_validation[i]-predictions[i])**2
    MSE = s/len(y)
    print(k,MSE)

0.01 0.43760267423644056
0.1 0.43735558819960496
1 0.4350170523269765
10 0.4199182162315345
100 0.43845429656950863


In [108]:
    #test,using 10
    X = [feature(d) for d in data_train]
    y = [d['review/overall'] for d in data_train]
    X_test = [feature(d) for d in data_test]
    y_test = [d['review/overall'] for d in data_test]

    clf = linear_model.Ridge(10, fit_intercept=False)
    clf.fit(X, y)
    theta = clf.coef_
    predictions = clf.predict(X_test)
    s= 0
    for i in range(len(y_test)):
        s +=(y_test[i]-predictions[i])**2
    MSE = s/len(y)
    print('10',MSE)

10 0.41616258466156125


In [ ]:
#so the matrix is 
#bigrams-removing-counts situation 1     regulization is 100 MSE 0.4932
#unigrams-removing-counts situation 2     regulization is 100 MSE 0.4269
#unigrams-removing-tfidf situation 3      regulization is 10   MSE 0.4461
#biograms-removing-tfidf situation 4      regulization  is 10   MSE 0.519
#biograms-not removing-tfidf situation 5   regulization is  10  MSE 0.49037
#uniograms-not removing-tfidf situation 6  too slow to get the result
#biograms-not removing-counts situation 7    regulization is 100  MSE is 0.4800
#uiograms-not removing-counts situation 8    regulization is 10  MSE is 0.4161626